In [ ]:
from google.colab import files

# Upload file from local PC
uploaded = files.upload()


Saving labeled_data.csv to labeled_data.csv


In [3]:
import pandas as pd

# Get the uploaded filename dynamically
filename = list(uploaded.keys())[0]

# Load CSV file
df = pd.read_csv(filename)

# Display first few rows
print(df.head())


   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [4]:
#preprocessing the data
import torch
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
df['tokenized'] = df['tweet'].apply(lambda x: tokenizer(x, padding='max_length', max_length=128, truncation=True, return_tensors="pt"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
#creating the data set for torch
from torch.utils.data import Dataset, DataLoader

class HateSpeechDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['tokenized']
        self.labels = dataframe['class'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor(self.labels[idx])

# Create dataset and dataloader
dataset = HateSpeechDataset(df)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [6]:
#fine-tuned bert for hate speech classification
from transformers import BertForSequenceClassification

# Load BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3 classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
from torch.optim import AdamW
from tqdm import tqdm
import torch

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Define loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0

    print(f"\nEpoch {epoch+1}/{epochs}")

    progress_bar = tqdm(dataloader, desc=f"Training Epoch {epoch+1}", leave=True)

    for step, batch in enumerate(progress_bar):
        inputs, labels = batch
        inputs = {key: val.squeeze().to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update progress bar with loss info
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} Completed | Average Loss: {avg_loss:.4f}")



Epoch 1/3


Training Epoch 1: 100%|██████████| 1549/1549 [08:32<00:00,  3.02it/s, loss=0.256]


Epoch 1 Completed | Average Loss: 0.2844

Epoch 2/3


Training Epoch 2: 100%|██████████| 1549/1549 [08:32<00:00,  3.02it/s, loss=0.293]


Epoch 2 Completed | Average Loss: 0.2068

Epoch 3/3


Training Epoch 3: 100%|██████████| 1549/1549 [08:32<00:00,  3.02it/s, loss=0.0208]

Epoch 3 Completed | Average Loss: 0.1574


In [8]:
torch.save(model.state_dict(), "hate_speech_model.pth")


In [9]:
def predict(text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=128, truncation=True).to(device)
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()

    classes = {0: "Hate Speech", 1: "Offensive Language", 2: "Neutral"}
    return classes[prediction]

# Example
print(predict(("fuck up")))  # Should detect hate speech or offensive language

Offensive Language


In [10]:

!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00


In [11]:
import torch
import gradio as gr

def predict(text):
    model.eval()
    tokens = tokenizer(text, return_tensors="pt", padding='max_length', max_length=128, truncation=True).to(device)
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()

    classes = {0: "Hate Speech", 1: "Offensive Language", 2: "Neutral"}
    return classes[prediction]

# Creating a Gradio Interface
iface = gr.Interface(fn=predict, inputs="text", outputs="text", title="Hate Speech/offensive Detection")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cada14146e02fbc315.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
